In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
import numpy as np
from nltk.corpus import words
nltk.download('words')
from nltk.corpus import brown
nltk.download('brown')
from numpy import genfromtxt
from tensorflow.keras import regularizers
import time
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pickle
import random
from kafka import KafkaConsumer
import time
import os
import sys

In [ ]:
from kafka import KafkaConsumer
import time
import os
import sys
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle

KAFKA_BOOTSTRAP_SERVER = "localhost:9092"
TOPIC = "tweets"

consumer = KafkaConsumer(
     TOPIC,
     bootstrap_servers=KAFKA_BOOTSTRAP_SERVER,
#      auto_offset_reset='earliest',
     enable_auto_commit=True)

CACHE_FILE = "cache{0}.txt"
TIME_INTERVEL = 60

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

while True:
    
    start_time = time.time()
    print(start_time)
    data = []
    
    with open(CACHE_FILE.format(count), "a") as f:
        for msg in consumer:
            msg = msg.value.decode("utf8")
            f.write(msg)
            data.append(msg)
            f.write("\n")
#             print(msg)
            if time.time() >= start_time + TIME_INTERVEL:
                ## load new model, make prediction
                model = tf.keras.models.load_model("./model")
                with open('trend.txt', 'rb') as f:
                    try:  # catch OSError in case of a one line file 
                        f.seek(-2, os.SEEK_END)
                        while f.read(1) != b'\n':
                            f.seek(-2, os.SEEK_CUR)
                    except OSError:
                        f.seek(0)
                    last_line = f.readline().decode()[:-1]
                    ts, label = last_line.split(",")
                label = int(label)

                # load model
                num_lines = len(data)
                loaded_model = tf.keras.models.load_model("./model")
                loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss=keras.losses.SparseCategoricalCrossentropy(),metrics=keras.metrics.SparseCategoricalAccuracy())
                labels_array = np.ones(num_lines) * label

                sequence = tokenizer.texts_to_sequences(data)
                seq_padded = pad_sequences(sequence, maxlen=500)
                pred = np.bincount(np.argmax(loaded_model.predict(seq_padded),axis=1)).argmax()
                
                print("predicted:  %s, actual: %s" % (pred, label))
                
                count = (count + 1) % 10
                
                
                break
